# 數據整理:join, combine, reshape

## 階層式索引
    - 分類欄或列

In [7]:
import numpy as np
import pandas as pd

data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                       [1, 2, 3, 1, 2, 3, 1, 2, 3]])
data
'''
Out[1]: 
a  1    0.205758
   2    0.914465
   3   -1.129890
b  1    0.841650
   2   -0.571143
c  3    0.215346
   1    0.426410
d  2    0.916470
   3   -1.792631
dtype: float64
'''

#分類化索引,方便透過分類,讓我們取得資料更方便
#標籤索引
data['b']
'''
Out[2]: 
1    0.841650
2   -0.571143
dtype: float64
'''

#標籤slice
data['b':'c']
'''
Out[3]: 
b  1    0.841650
   2   -0.571143
c  3    0.215346
   1    0.426410
dtype: float64
'''

#loc[list]
data.loc[['b', 'd']]
'''
Out[4]: 
b  1    0.841650
   2   -0.571143
d  2    0.916470
   3   -1.792631
dtype: float64
'''

#loc[外部標籤,內部標籤]
data[:,2]
'''
Out[5]: 
a    0.914465
b   -0.571143
d    0.916470
dtype: float64
'''

#階層式索引可以運用在重新塑型資料和使用群組運算
#unstack()
data.unstack()
'''
Out[6]: 
          1         2         3
a  0.205758  0.914465 -1.129890
b  0.841650 -0.571143       NaN
c  0.426410       NaN  0.215346
d       NaN  0.916470 -1.792631
'''

,1,2,3
a,-0.394553,-1.221037,-0.626661
b,1.819467,0.567475,NaN
c,-0.202962,NaN,-0.229614
d,NaN,-0.900836,0.411256


In [13]:
import numpy as np
import pandas as pd

frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a','a','b','b'],[1, 2, 1, 2]],
                    columns=[['台北','台北','台中'],['Green','Red','Green']])
frame
'''
Out[7]: 
       台北        台中
    Green Red Green
a 1     0   1     2
  2     3   4     5
b 1     6   7     8
  2     9  10    11
'''

#階層式的索引可以命名
frame.index.names = ['key1', 'key2']
frame.columns.names = ['縣市','顏色']
frame
'''
Out[8]: 
縣市           台北        台中
顏色        Green Red Green
key1 key2                
a    1        0   1     2
     2        3   4     5
b    1        6   7     8
     2        9  10    11
'''

#欄位索引
frame['台北']
'''
Out[9]: 
顏色         Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10
'''

#可以使用MultiIndex可以獨自建立
pd.MultiIndex.from_arrays([['台北','台北','台中'],['Green','Red','Green']],
                       names=['縣市','顏色'])
'''
Out[10]: 
MultiIndex([('台北', 'Green'),
            ('台北',   'Red'),
            ('台中', 'Green')],
           names=['縣市', '顏色'])
'''

MultiIndex([('台北', 'Green'),
            ('台北',   'Red'),
            ('台中', 'Green')],
           names=['縣市', '顏色'])

### 安排位置和排序
- 重新安排階層位置和依內容排序

In [21]:
import numpy as np
import pandas as pd

frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a','a','b','b'],[1, 2, 1, 2]],
                    columns=[['台北','台北','台中'],['Green','Red','Green']])
frame.index.names = ['key1', 'key2']
frame.columns.names = ['縣市','顏色']
frame
'''
Out[11]: 
縣市           台北        台中
顏色        Green Red Green
key1 key2                
a    1        0   1     2
     2        3   4     5
b    1        6   7     8
     2        9  10    11
'''

#使用swaplevel()對調索引
frame.swaplevel('key1','key2')
'''
Out[12]: 
縣市           台北        台中
顏色        Green Red Green
key2 key1                
1    a        0   1     2
2    a        3   4     5
1    b        6   7     8
2    b        9  10    11
'''
#使用sort_index(),依索引內容排序
frame.sort_index(level=0)
'''
Out[13]: 
縣市           台北        台中
顏色        Green Red Green
key1 key2                
a    1        0   1     2
     2        3   4     5
b    1        6   7     8
     2        9  10    11
'''

frame.sort_index(level=1)
'''
Out[14]: 
縣市           台北        台中
顏色        Green Red Green
key1 key2                
a    1        0   1     2
b    1        6   7     8
a    2        3   4     5
b    2        9  10    11


'''

frame.swaplevel(0,1).sort_index(level=0)
'''
Out[15]: 
縣市           台北        台中
顏色        Green Red Green
key2 key1                
1    a        0   1     2
     b        6   7     8
2    a        3   4     5
     b        9  10    11
'''

縣市           台北        台中
顏色        Green Red Green
key2 key1                
1    a        0   1     2
     b        6   7     8
2    a        3   4     5
     b        9  10    11

### 透過level的統計

In [24]:
frame
'''
Out[16]: 
縣市           台北        台中
顏色        Green Red Green
key1 key2                
a    1        0   1     2
     2        3   4     5
b    1        6   7     8
     2        9  10    11
'''

frame.sum(level='key1')
'''
Out[17]: 
縣市      台北        台中
顏色   Green Red Green
key1                
a        3   5     7
b       15  17    19
'''

frame.sum(level='顏色', axis=1)
'''
Out[18]: 
顏色         Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10
'''



顏色         Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10